요약 from https://databuzz-team.github.io/2018/11/11/make_pipeline/

Scikit-Learn은 덕 타이핑duck typing을 지원하므로 fit(), transform(), fit_transform() 메서드를 구현한 파이썬 클래스를 만들면 됩니다. 여기에서 덕 타이핑이란 상속이나 인터페이스 구현이 아니라 객체의 속성이나 메서드가 객체의 유형을 결정하는 방식입니다. 
- BaseEstimator를 상속하면 하이퍼파라미터 튜닝에 필요한 두 메서드 get_params()와 set_params()를 얻게 됩니다. cross_val_score, gridsearchcv 등을 만들때도 사용. 이때 생성자에 \*args나 \*\*kargs를 사용하지 않아야 합니다.
- 마지막의 fit_transform()은 TransformerMixin으로 구현이 되어 있고 이를 상속하면 자동으로 생성됩니다

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [81]:
train_df = pd.read_csv('https://raw.githubusercontent.com/fasthill/My-gist/main/data/titanic/train.csv')

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [12]:
train_df.select_dtypes('number').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          714 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


In [13]:
train_df.select_dtypes('object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      891 non-null    object
 1   Sex       891 non-null    object
 2   Ticket    891 non-null    object
 3   Cabin     204 non-null    object
 4   Embarked  889 non-null    object
dtypes: object(5)
memory usage: 34.9+ KB


In [55]:
class AgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X['Age'].fillna(X['Age'].mean(), inplace=True)
        X['Age'] = X['Age'].astype('int')
        return X

In [69]:
age_transform = AgeTransformer()
age_transform.fit_transform(train_df).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


위 코드와 같이 용도에 따라서 fit(), transform()을 만들어 주고 TransformerMixin을 상속해주기만 하면 fit_transform()이 생성됩니다. 여러가지의 변환기를 연결시켜주는 Pipeline을 만들기 위해 name값을 first name만 표시하는 변환기를 만들어 보겠습니다.

In [57]:
class NameTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.name_ls = []
        return
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        for name_idx in range(len(X['Name'])):
            self.name_ls.append(X['Name'][name_idx].split(',')[0])
        X['Name'] = self.name_ls
        return X

In [70]:
name_transform = NameTransformer()
name_transform.fit_transform(train_df).head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund,male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings,female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle,female,35,1,0,113803,53.1000,C123,S
4,5,0,3,Allen,male,35,0,0,373450,8.0500,NaN,S


In [82]:
titanic_pipeline = Pipeline([
    ('age_transform', AgeTransformer()),
    ('name_transform', NameTransformer())
])

In [ ]:
train_df = pd.read_csv('https://raw.githubusercontent.com/fasthill/My-gist/main/data/titanic/train.csv')

In [83]:
trans_train = titanic_pipeline.fit_transform(train_df)
trans_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund,male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings,female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle,female,35,1,0,113803,53.1000,C123,S
4,5,0,3,Allen,male,35,0,0,373450,8.0500,NaN,S


사이킷런의 FeatureUnion을 사용하면 여러개의 pipeline을 하나의 pipeline으로 합칠 수도 있습니다. 하지만 아직 사이킷런의 pipeline에는 Pandas의 DataFrame을 직접 주입할 수 없고 결과도 array로 반환되기 때문에 이를 염두에 두고 만들어야 합니다. 필요에 따라서는 필요한 특성만을 선택하는 변환기를 따로 만들어야 하는 경우도 있습니다. 처음에는 어려울 수도 있지만 간단한 것부터 만들면서 원리를 익혀간다면 필요한 변환기를 만들 수 있을 것입니다.